### Importing Libraries
The first step in any Python script or Jupyter notebook is to import the necessary libraries. Here, we are importing libraries that will be used for stock data retrieval, risk portfolio analysis, time management, threading, and interaction with the Interactive Brokers API.

In [1]:
import time
import threading
import numpy as np
import pandas as pd
import datetime
from openbb_terminal.sdk import openbb
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

In [2]:
# Set the values we'll use in the trading app
today = pd.Timestamp.today().strftime("%Y-%m-%d")
top_N = 5

### Universe Selection
In this section, we are using the openbb_terminal.sdk library to screen for stocks that have recently hit new highs. This is as much art as science. Use your skill and experience to select your investment universe. Ideally a group of stocks you think are showing signs of momentum.

In [3]:
new_highs = openbb.stocks.screener.screener_data("new_high")

### Data Filtering
After retrieving the stock data, we filter it based on certain criteria. Here, we are filtering for stocks with a price greater than $15 and that are based in the USA.

In [4]:
port_data = new_highs[
    (new_highs.Price > 15) &
    (new_highs.Country == "USA")
]

### Data Preparation
Once we have our filtered list of stocks, we extract their ticker symbols and then use these symbols to download historical stock price data. This data will be used for further analysis.

In [5]:
symbols = port_data.Ticker.tolist()

In [6]:
symbols

['CYTK',
 'MARA',
 'MSTR',
 'ASPN',
 'COIN',
 'REFI',
 'BBIO',
 'CBAY',
 'ACU',
 'LMNR',
 'AGM',
 'ITCI',
 'MGRC',
 'RCKT',
 'CVRX',
 'STRL',
 'GFF',
 'MMI',
 'CUBI',
 'WD',
 'BHVN',
 'FRPT',
 'RYTM',
 'SWTX',
 'KD',
 'LMB',
 'HLNE',
 'COKE',
 'FSTR',
 'ARVN',
 'NWSA',
 'NWS',
 'RXST',
 'MHO',
 'AMD',
 'VSEC',
 'WS',
 'BPMC',
 'SKYW',
 'NPO',
 'SPXC',
 'INTC',
 'CVCO',
 'HBB',
 'INSM',
 'BCC',
 'BXC',
 'UVV',
 'CABA',
 'CCS',
 'ML',
 'REGN',
 'UHAL-B',
 'ANDE',
 'EVR',
 'DLX',
 'UHAL',
 'GHM',
 'EAT',
 'RRR',
 'BFST',
 'SLM',
 'AFRM',
 'NBIX',
 'ODP',
 'MDC',
 'TDW',
 'CRVL',
 'OVLY',
 'STC',
 'CENT',
 'GPI',
 'LOB',
 'GOLF',
 'TPH',
 'BWMN',
 'PJT',
 'ZS',
 'PCVX',
 'FUNC',
 'WMS',
 'BX',
 'JLL',
 'SCSC',
 'WFRD',
 'KOP',
 'DSGR',
 'PATK',
 'BBSI',
 'RAMP',
 'SSD',
 'BR',
 'VSTS',
 'ALG',
 'FNF',
 'RUSHA',
 'BCPC',
 'GNE',
 'USAP',
 'MEDP',
 'SKY',
 'SCVL',
 'CHCO',
 'JELD',
 'MTRN',
 'MSBI',
 'KBH',
 'WING',
 'NCNO',
 'MLI',
 'IBOC',
 'ZEUS',
 'ICE',
 'KELYA',
 'RKT']

In [7]:
# Using an arbitrary list of stocks for demonstration purposes. This
# Overrides the stock universe created above.
# symbols = ["NEM", "RGLD", "SSRM", "CDE", "LLY", "UNH", "JNJ", "MRK"]
stocks = []
for symbol in symbols:
    df = (
        openbb
        .stocks
        .load(
            symbol,
            start_date="2015-01-01", 
            end_date=today,
            verbose=False
        )
        .drop(["Close", "Dividends", "Stock Splits"], axis=1)
    )
    df["symbol"] = symbol
    stocks.append(df)

prices = pd.concat(stocks)
prices.columns = ["open", "high", "low", "close", "volume", "symbol"]

### Factor Engineering
With the historical stock price data in hand, we can now proceed to engineer our momentum factor.

In [8]:
# Check to make sure we have at least 2 years worth of data
nobs = prices.groupby("symbol").size()
mask = nobs[nobs > 2 * 12 * 21].index
prices = prices[prices.symbol.isin(mask)]

In [9]:
prices

,open,high,low,close,volume,symbol
date,,,,,,
2015-01-02,8.01,8.16,7.14,7.37,1675600,CYTK
2015-01-05,7.50,7.98,7.50,7.67,730800,CYTK
2015-01-06,7.70,7.84,6.92,7.22,979700,CYTK
2015-01-07,7.25,7.50,7.16,7.47,546200,CYTK
2015-01-08,7.50,8.07,7.49,7.72,892200,CYTK
...,...,...,...,...,...,...
2023-12-20,14.39,14.97,14.18,14.30,9486400,RKT
2023-12-21,14.52,14.96,14.42,14.90,4865000,RKT
2023-12-22,14.87,15.00,14.60,14.73,3046100,RKT


In [10]:
# Arrange the DataFrame to have a symbol and date index
prices = (
    prices
    .set_index("symbol", append=True)
    .reorder_levels(["symbol", "date"])
).drop_duplicates()

In [11]:
prices

open   high    low  close   volume
symbol date                                           
CYTK   2015-01-02   8.01   8.16   7.14   7.37  1675600
       2015-01-05   7.50   7.98   7.50   7.67   730800
       2015-01-06   7.70   7.84   6.92   7.22   979700
       2015-01-07   7.25   7.50   7.16   7.47   546200
       2015-01-08   7.50   8.07   7.49   7.72   892200
...                  ...    ...    ...    ...      ...
RKT    2023-12-20  14.39  14.97  14.18  14.30  9486400
       2023-12-21  14.52  14.96  14.42  14.90  4865000
       2023-12-22  14.87  15.00  14.60  14.73  3046100
       2023-12-26  14.75  15.08  14.62  14.91  1909500
       2023-12-27  14.91  15.19  14.83  15.04  1869100

[221635 rows x 5 columns]

In [12]:
# The function first computes the percentage change in the 
# closing prices over the most recent 126 trading days, 
# which is used later for normalization. The momentum 
# score is then calculated as the difference between 
# the long-term return (the percentage change in closing 
# price over the past 252 days) and the short-term return 
# (the percentage change in closing price over the past 21
# days). This score is normalized by dividing it by the 
# standard deviation of the returns over the past 126 days, 
# providing a standardized measure of momentum that accounts 
# for recent volatility. 
def momentum(close):
    returns = close.pct_change()[-126:]
    return(
        (close[-21] - close[-252]) / close[-252] 
        - (close[-1] - close[-21]) / close[-21]
    ) / np.std(returns)

In [13]:
# Apply the momentum factor to a 252 trading day rolling
# window to generate the momentum factor
df = (
    prices
    .groupby(
        "symbol", 
        group_keys=False
    )
    .rolling(252)
    .close
    .apply(momentum)
)

In [14]:
# Pandas adds an extra symbol index, so remove it
df.index = df.index.droplevel(0)

In [15]:
# Add our momentum factor values to our original DataFrame
# and drop any NA values
prices["momentum"] = df
prices.dropna(inplace=True)

In [16]:
prices

open   high    low  close   volume   momentum
symbol date                                                      
CYTK   2015-12-31  10.49  10.64  10.26  10.46   600200  21.555587
       2016-01-04  10.22  10.34   9.85  10.00   403000  13.528984
       2016-01-05  10.09  10.71   9.95  10.60   540300  14.039476
       2016-01-06  10.44  10.44   9.86   9.94   375200  13.016054
       2016-01-07   9.77  10.10   9.55   9.82   445900  15.389432
...                  ...    ...    ...    ...      ...        ...
RKT    2023-12-20  14.39  14.97  14.18  14.30  9486400 -12.373634
       2023-12-21  14.52  14.96  14.42  14.90  4865000 -13.944638
       2023-12-22  14.87  15.00  14.60  14.73  3046100 -11.968475
       2023-12-26  14.75  15.08  14.62  14.91  1909500 -10.829116
       2023-12-27  14.91  15.19  14.83  15.04  1869100  -7.366365

[193774 rows x 6 columns]

In [17]:
# Rank the momentum factor for each stock, every day
prices["factor_rank"] = (
    prices
    .groupby(level=[1])
    .momentum
    .rank(ascending=False)
)

In [18]:
prices

open   high    low  close   volume   momentum  factor_rank
symbol date                                                                   
CYTK   2015-12-31  10.49  10.64  10.26  10.46   600200  21.555587         12.0
       2016-01-04  10.22  10.34   9.85  10.00   403000  13.528984         22.0
       2016-01-05  10.09  10.71   9.95  10.60   540300  14.039476         23.0
       2016-01-06  10.44  10.44   9.86   9.94   375200  13.016054         22.0
       2016-01-07   9.77  10.10   9.55   9.82   445900  15.389432         17.0
...                  ...    ...    ...    ...      ...        ...          ...
RKT    2023-12-20  14.39  14.97  14.18  14.30  9486400 -12.373634        103.0
       2023-12-21  14.52  14.96  14.42  14.90  4865000 -13.944638        104.0
       2023-12-22  14.87  15.00  14.60  14.73  3046100 -11.968475        101.0
       2023-12-26  14.75  15.08  14.62  14.91  1909500 -10.829116         95.0
       2023-12-27  14.91  15.19  14.83  15.04  1869100  -7.366365         87.0

[193774 rows x 7 columns]

In [19]:
prices.reset_index().groupby('symbol').apply(lambda x: x.loc[x['date'].idxmax()])

,symbol,date,open,high,low,close,volume,momentum,factor_rank
symbol,,,,,,,,,
ACU,ACU,2023-12-27,43.200001,44.860001,43.200001,44.860001,12400,8.284886,47.0
AFRM,AFRM,2023-12-27,50.200001,52.480000,49.799999,51.240002,20755800,35.393350,12.0
AGM,AGM,2023-12-27,192.000000,198.169998,190.699997,194.919998,74400,15.889454,28.0
ALG,ALG,2023-12-27,214.000000,216.339996,211.410004,213.250000,46400,6.705429,54.0
AMD,AMD,2023-12-27,144.720001,146.250000,143.179993,146.070007,48984900,28.081430,17.0
...,...,...,...,...,...,...,...,...,...
WFRD,WFRD,2023-12-27,101.260002,102.644997,100.519997,100.680000,696700,34.732896,13.0
WING,WING,2023-12-27,259.149994,261.970001,258.609985,260.390015,180600,27.790784,18.0
WMS,WMS,2023-12-27,144.639999,145.679993,143.445007,143.800003,345200,13.451969,34.0


In [20]:
# Generate the portfolio we want to trade based on the Top
# N stocks we want to trade
stocks_to_trade = (
    prices.reset_index()
    .groupby('symbol')
    .apply(lambda x: x.loc[x['date'].idxmax()])
    .sort_values("factor_rank")
    .head(top_N)
)

In [21]:
stocks_to_trade

,symbol,date,open,high,low,close,volume,momentum,factor_rank
symbol,,,,,,,,,
SKYW,SKYW,2023-12-27,52.000000,53.130001,51.910000,52.610001,301500,75.623469,1.0
LMB,LMB,2023-12-27,44.799999,46.320000,44.530998,46.110001,154100,58.977474,2.0
MSTR,MSTR,2023-12-27,613.799988,673.820007,612.010010,670.710022,2117100,56.677805,3.0
COIN,COIN,2023-12-27,176.320007,186.970001,175.500000,185.240005,15482900,55.487895,4.0
FSTR,FSTR,2023-12-27,21.200001,22.559999,21.200001,22.320000,52800,51.099499,5.0


### Compute the number of shares to purchase
Compute the number of shares to purchase based on the optimized weights

In [22]:
port_val = 60_000

In [23]:
stocks_to_trade

,symbol,date,open,high,low,close,volume,momentum,factor_rank
symbol,,,,,,,,,
SKYW,SKYW,2023-12-27,52.000000,53.130001,51.910000,52.610001,301500,75.623469,1.0
LMB,LMB,2023-12-27,44.799999,46.320000,44.530998,46.110001,154100,58.977474,2.0
MSTR,MSTR,2023-12-27,613.799988,673.820007,612.010010,670.710022,2117100,56.677805,3.0
COIN,COIN,2023-12-27,176.320007,186.970001,175.500000,185.240005,15482900,55.487895,4.0
FSTR,FSTR,2023-12-27,21.200001,22.559999,21.200001,22.320000,52800,51.099499,5.0


In [24]:
stocks_to_trade["invest_amt"] = (1 / top_N) * port_val

In [25]:
stocks_to_trade

,symbol,date,open,high,low,close,volume,momentum,factor_rank,invest_amt
symbol,,,,,,,,,,
SKYW,SKYW,2023-12-27,52.000000,53.130001,51.910000,52.610001,301500,75.623469,1.0,12000.0
LMB,LMB,2023-12-27,44.799999,46.320000,44.530998,46.110001,154100,58.977474,2.0,12000.0
MSTR,MSTR,2023-12-27,613.799988,673.820007,612.010010,670.710022,2117100,56.677805,3.0,12000.0
COIN,COIN,2023-12-27,176.320007,186.970001,175.500000,185.240005,15482900,55.487895,4.0,12000.0
FSTR,FSTR,2023-12-27,21.200001,22.559999,21.200001,22.320000,52800,51.099499,5.0,12000.0


In [26]:
stocks_to_trade["shares"] = (stocks_to_trade.invest_amt / stocks_to_trade.close).astype(int)

In [27]:
(stocks_to_trade.shares * stocks_to_trade.close).sum()

59226.950859069824

### Execute the orders on IB
Set up the IB app, contracts, and order submission. Connect to the app and execute orders through the API.

In [28]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
    
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.nextOrderId = orderId

In [29]:
def stock_contract(symbol, secType="STK", exchange="SMART", currency="USD"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = secType
    contract.exchange = exchange
    contract.currency = currency

    return contract

def submit_order(contract, direction, qty=100, orderType="MKT", transmit=True):
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = orderType
    order.transmit = transmit
    order.eTradeOnly = ""
    order.firmQuoteOnly = ""
    # submit order
    app.placeOrder(app.nextOrderId, contract, order)
    app.nextOrderId += 1

In [30]:
def run_loop():
    app.run()

app = IBapi()
app.connect('127.0.0.1', 7497, 123)
app.nextOrderId = None

api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

while True:
    if isinstance(app.nextOrderId, int):
        print("Connected")
        break
    else:
        print("Waiting")
        time.sleep(1)

Waiting
Connected


In [31]:
for row in stocks_to_trade.itertuples():
    contract = stock_contract(row.Index)
    submit_order(contract, direction="BUY", qty=row.shares)